In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers.legacy import Adam


In [2]:
# Directorios
data_dir = '/Users/baudi/AI/practicas/uvas/data'  # Carpeta principal de datos
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [3]:
# Configuración de los generadores de imágenes
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                  shuffle=False)

Found 2843 images belonging to 4 classes.
Found 611 images belonging to 4 classes.


In [4]:
# Creación del modelo
base_model = MobileNetV2(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-04-02 18:56:15.980002: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-02 18:56:15.980226: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
# Entrenamiento del modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_generator, epochs=5, validation_data=test_generator)


Epoch 1/5


2023-04-02 18:56:18.493060: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-04-02 18:56:20.004702: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


89/89 [==============================] - ETA: 0s - loss: 0.2292 - accuracy: 0.9388

2023-04-02 18:56:39.444289: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


89/89 [==============================] - 22s 217ms/step - loss: 0.2292 - accuracy: 0.9388 - val_loss: 0.5765 - val_accuracy: 0.9558
Epoch 2/5
89/89 [==============================] - 16s 182ms/step - loss: 0.1276 - accuracy: 0.9669 - val_loss: 9.4179 - val_accuracy: 0.5074
Epoch 3/5
89/89 [==============================] - 16s 179ms/step - loss: 0.0559 - accuracy: 0.9863 - val_loss: 3.8021 - val_accuracy: 0.6628
Epoch 4/5
89/89 [==============================] - 16s 182ms/step - loss: 0.0574 - accuracy: 0.9845 - val_loss: 4.2752 - val_accuracy: 0.7561
Epoch 5/5
89/89 [==============================] - 16s 181ms/step - loss: 0.0687 - accuracy: 0.9842 - val_loss: 9.4843 - val_accuracy: 0.7218


In [6]:
# Matriz de confusión y métricas adicionales
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)

print('Classification Report:')
print(classification_report(test_generator.classes, y_pred, target_names=test_generator.class_indices.keys()))

print('Confusion Matrix:')
print(confusion_matrix(test_generator.classes, y_pred))

2023-04-02 18:58:14.504712: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


20/20 [==============================] - 2s 51ms/step
Classification Report:
                            precision    recall  f1-score   support

         Grape___Black_rot       0.51      1.00      0.68       177
Grape___Esca_Black_Measles       0.98      0.62      0.76       208
       Grape___Leaf_blight       1.00      0.83      0.91       162
           Grape___healthy       0.00      0.00      0.00        64

                  accuracy                           0.72       611
                 macro avg       0.62      0.61      0.59       611
              weighted avg       0.75      0.72      0.70       611

Confusion Matrix:
[[177   0   0   0]
 [ 78 130   0   0]
 [ 26   2 134   0]
 [ 64   0   0   0]]


/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/baudi/miniconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
